In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/song_lyrics_en.csv')

df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,popularity_bin,lyrics_cleaned,feature_cleaned,song_document
0,0,0,0,0,Killa Cam,rap,Cam'ron,2004,173166,Cam\\ron Opera Steve,"Killa Cam, Killa Cam, Cam\nKilla Cam, Killa Ca...",1,en,en,en,Medium,"Killa Cam, Killa Cam, Cam\nKilla Cam, Killa Ca...",Cam\\ron Opera Steve,"Killa Cam Killa Cam, Killa Cam, Cam\nKilla Cam..."
1,1,1,1,1,Can I Live,rap,JAY-Z,1996,468624,NaN,"Yeah, hah, yeah, Roc-A-Fella\nWe invite you to...",3,en,en,en,Medium,"Yeah, hah, yeah, Roc-A-Fella\nWe invite you to...",NaN,"Can I Live Yeah, hah, yeah, Roc-A-Fella\nWe in..."
2,2,2,2,2,Forgive Me Father,rap,Fabolous,2003,4743,NaN,Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en,Very Low,Maybe cause I'm eatin\nAnd these bastards fien...,NaN,Forgive Me Father Maybe cause I'm eatin\nAnd t...
3,3,3,3,3,Down and Out,rap,Cam'ron,2004,144404,Cam\\ron Kanye West Syleena Johnson,"Ugh, Killa!\nBaby!\nKanye, this that 1970s Her...",5,en,en,en,Medium,"Ugh, Killa!\nBaby!\nKanye, this that 1970s Her...",Cam\\ron Kanye West Syleena Johnson,"Down and Out Ugh, Killa!\nBaby!\nKanye, this t..."
4,4,4,4,4,Fly In,rap,Lil Wayne,2005,78271,NaN,"So they ask me\n""Young boy\nWhat you gon' do t...",6,en,en,en,Low,"So they ask me\n""Young boy\nWhat you gon' do t...",NaN,"Fly In So they ask me\n""Young boy\nWhat you go..."


In [3]:
slice_num = 10000
df = df[:slice_num]

In [4]:
import spacy
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_pandas

tqdm_pandas(tqdm())

nlp = spacy.load('en_core_web_md')
tag_embeddings = np.load('data/tag_embeddings.npy')
artist_embeddings = np.load('data/artist_embeddings.npy')

tag_embeddings

0it [00:00, ?it/s]


array([[[-0.04503907,  0.0417758 ,  0.01997494]],

       [[-0.04503907,  0.0417758 ,  0.01997494]],

       [[-0.04503907,  0.0417758 ,  0.01997494]],

       ...,

       [[-0.03446241,  0.01993293,  0.01982058]],

       [[-0.03446241,  0.01993293,  0.01982058]],

       [[ 0.00429112,  0.00109095,  0.01435422]]], dtype=float32)

In [8]:
artist_embeddings.shape

(3374198, 1, 600)

In [6]:
tag_embeddings = tag_embeddings[:, 0, :]
tag_embeddings = tag_embeddings[:slice_num]
tag_embeddings

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [9]:
artist_embeddings = artist_embeddings[:, 0, :]
artist_embeddings = artist_embeddings[:slice_num]
artist_embeddings

array([[-0.03651663, -0.04300829, -0.00096111, ...,  0.04577483,
         0.03669837, -0.01590056],
       [ 0.00797558,  0.01449763,  0.02677281, ...,  0.01654521,
         0.02038094,  0.02224332],
       [ 0.02283705,  0.04656899, -0.03825787, ...,  0.01336754,
        -0.03642181, -0.01813016],
       ...,
       [ 0.00858756, -0.04817573,  0.0425526 , ..., -0.02735157,
         0.03027593,  0.00185962],
       [ 0.00858756, -0.04817573,  0.0425526 , ..., -0.02735157,
         0.03027593,  0.00185962],
       [ 0.00858756, -0.04817573,  0.0425526 , ..., -0.02735157,
         0.03027593,  0.00185962]], dtype=float32)

In [10]:
artist_embeddings.shape

(10000, 600)

In [11]:
def get_word_vector(text):
    doc = nlp(text)
    return np.mean([token.vector for token in doc if token.has_vector], axis=0)

def transform_y(label):
    if label == 'Very Low':
        return 0
    if label == 'Low':
        return 1
    if label == 'Medium':
        return 2
    if label == 'High':
        return 3
    if label == 'Very High':
        return 4

def f1_score(y_true, y_pred):
    precision = tf.keras.metrics.Precision()(y_true, y_pred)
    recall = tf.keras.metrics.Recall()(y_true, y_pred)
    return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())



df['vector'] = df['song_document'].progress_apply(get_word_vector)

X = np.vstack(df['vector'].values)  



10000it [49:42,  3.35it/s]


In [12]:
np.save('data/song_document_w2v.npy', X)

In [13]:
X = np.hstack([X, tag_embeddings, artist_embeddings])


KeyError: 'label'

In [16]:
y = df['popularity_bin'].apply(transform_y).values

In [18]:
X.shape

(10000, 903)

In [19]:
y.shape

(10000,)

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Assuming X and y are your input and output arrays
# X.shape = (10000, 903), y.shape = (10000,)

# Convert y to one-hot encoded format for multi-class classification
y_one_hot = to_categorical(y, num_classes=5)
y_one_hot

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)


In [32]:
y_train.shape

(8000, 5)

In [38]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    y_pred_classes = K.argmax(y_pred, axis=-1)
    y_true_classes = K.argmax(y_true, axis=-1)
    
    precision = tf.metrics.Precision()(y_true_classes, y_pred_classes)
    recall = tf.metrics.Recall()(y_true_classes, y_pred_classes)
    
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1

In [40]:
model1 = Sequential()

# Input layer and first hidden layer (Dense)
model1.add(Dense(128, input_dim=903, activation='relu'))  # Input dim matches the number of features in X

# Add additional hidden layers as needed
model1.add(Dense(64, activation='relu'))

# Output layer (for 5 classes)
model1.add(Dense(5, activation='softmax'))  # Softmax for multi-class classification


model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

loss, accuracy = model1.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


Epoch 1/20
200/200 [==============================] - 0s 1ms/step - loss: 0.9968 - accuracy: 0.5880 - val_loss: 0.8789 - val_accuracy: 0.6425
Epoch 2/20
200/200 [==============================] - 0s 729us/step - loss: 0.8144 - accuracy: 0.6708 - val_loss: 0.8117 - val_accuracy: 0.6756
Epoch 3/20
200/200 [==============================] - 0s 725us/step - loss: 0.7605 - accuracy: 0.6919 - val_loss: 0.8483 - val_accuracy: 0.6450
Epoch 4/20
200/200 [==============================] - 0s 721us/step - loss: 0.7327 - accuracy: 0.7045 - val_loss: 0.7974 - val_accuracy: 0.6756
Epoch 5/20
200/200 [==============================] - 0s 715us/step - loss: 0.7038 - accuracy: 0.7133 - val_loss: 0.8245 - val_accuracy: 0.6694
Epoch 6/20
200/200 [==============================] - 0s 715us/step - loss: 0.6939 - accuracy: 0.7161 - val_loss: 0.8023 - val_accuracy: 0.6775
Epoch 7/20
200/200 [==============================] - 0s 708us/step - loss: 0.6703 - accuracy: 0.7284 - val_loss: 0.8011 - val_accuracy: 0

In [41]:
from sklearn.metrics import f1_score
import numpy as np

# Get predictions from the model (these will be probabilities)
y_pred_prob = model1.predict(X_test)

# Convert probabilities to class predictions (e.g., the index of the max value)
y_pred = np.argmax(y_pred_prob, axis=1)

# If y_test is one-hot encoded, convert it back to class labels
y_test_class = np.argmax(y_test, axis=1)

# Calculate the F1 score using sklearn
f1 = f1_score(y_test_class, y_pred, average='weighted')

# Print the F1 score
print(f"Test F1 Score: {f1:.2f}")


63/63 [==============================] - 0s 369us/step
Test F1 Score: 0.63
